<h1><b>RECOMENDABLE EJECUTAR EN KERNEL PYTHON 3.12</b>

Importes necesarios</h1>

In [111]:
'''
%pip install --upgrade pip
%pip install scikit-learn
%pip install pandas
%pip install numpy
%pip install joblib
%pip install tensorflow[and-cuda]
%pip install keras
%pip install xgboost
'''

'\n%pip install --upgrade pip\n%pip install scikit-learn\n%pip install pandas\n%pip install numpy\n%pip install joblib\n%pip install tensorflow[and-cuda]\n%pip install keras\n%pip install xgboost\n'

In [112]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf
import time
import keras
import joblib
import os
import random
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from preprocessing import *

<h1>Pipeline de limpieza y preparación de datos</h1>

In [113]:
preprocesador = Pipeline([
    ('limpiador', Limpiador()),
    ('imputador', SimpleImputer(strategy='constant', fill_value=0)),
    ('escalador', StandardScaler())
])

<h1>Ejecutar el pipeline sobre el set de datos y exportar</h1>

In [114]:
df = pd.read_csv("muestra_ataques.csv", header=None)
X = df.iloc[:, 1:-1]
X = preprocesador.fit_transform(X)
y = df.iloc[:, -1].str.strip(" '").replace({'Normal': 0, 'normal': 0, 'Ataque': 1, 'ataque': 1}).fillna(0).to_numpy(dtype=np.float32)
joblib.dump(preprocesador, "preprocesador.pkl")

['preprocesador.pkl']

<h1>Definir set de entrenamiento del 80% y set de prueba del 20%</h1>

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
modelos = []

<h1>Función para prueba de modelos</h1>

In [116]:
def test_model(nombre, modelo, X_test, y_test, report=True):
    inicio = time.time()

    if hasattr(modelo, 'n_features_in_') and modelo.n_features_in_ < X_test.shape[1]:
        y_hat = (modelo.predict(PCA(n_components=modelo.n_features_in_).fit_transform(X_test))>=0.5).astype(np.float32)
    else:
        y_hat = (modelo.predict(X_test)>=0.5).astype(np.float32)

    ejecucion = time.time() - inicio

    if report:
        print('\n' + '-'*30 + f' {nombre} ' + '-'*30)
        print('\nExactitud:', accuracy_score(y_test,y_hat))
        print(f'Tiempo promedio de inferencia: {ejecucion*10e3 : .5f} ms')
        print('\nMatriz de confusion:\n', confusion_matrix(y_test, y_hat))
        print('\nReporte:\n', classification_report(y_test, y_hat))
    return [nombre, modelo, f1_score(y_test, y_hat), ejecucion]

<h1>Búsqueda de hiperparámetros para modelo Logistic Regression con validación cruzada de 3 carpetas usando concurrencia</h1>

In [117]:
parametros = {
    'solver': ['saga'],
    'l1_ratio': [0, 0.1, 0.3, 0.5, 0.8, 1],
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced', None]
}

modelo = LogisticRegression(random_state=123)

grid_search = GridSearchCV(modelo, parametros, cv=3, scoring='f1', n_jobs=-1, verbose=1)

inicio = time.time()

grid_search = grid_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('lr', grid_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Tiempo de ejecucion:  0.96 minutos

------------------------------ lr ------------------------------

Exactitud: 0.9977867401141058
Tiempo promedio de inferencia:  10.05411 ms

Matriz de confusion:
 [[20060    14]
 [   31   227]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       0.94      0.88      0.91       258

    accuracy                           1.00     20332
   macro avg       0.97      0.94      0.95     20332
weighted avg       1.00      1.00      1.00     20332



c:\Users\ismae\AppData\Local\Python\pythoncore-3.12-64\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


<h1>Búsqueda aleatoria de hiperparámetros para modelo Decision Tree con validación cruzada de 3 carpetas usando concurrencia</h1>

In [118]:
modelo = DecisionTreeClassifier(random_state=123)

parametros = {'max_depth': [i for i in range(1,13)]+[None],
              'criterion': ['gini','entropy'],
              'min_samples_split': [i for i in range(2,9)],
              'min_samples_leaf': [i for i in range(1,7)],
              'ccp_alpha': [0,0.0001,0.001,0.01,0.1,1],
              'class_weight': ['balanced', None]
             }

random_search = RandomizedSearchCV(modelo, parametros, cv=3, random_state=123, n_iter=2000, scoring='f1', n_jobs=-1, verbose=1)
inicio = time.time()

random_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('clf', random_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 2000 candidates, totalling 6000 fits
Tiempo de ejecucion:  1.89 minutos

------------------------------ clf ------------------------------

Exactitud: 0.999606531575841
Tiempo promedio de inferencia:  29.99067 ms

Matriz de confusion:
 [[20073     1]
 [    7   251]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       1.00      0.97      0.98       258

    accuracy                           1.00     20332
   macro avg       1.00      0.99      0.99     20332
weighted avg       1.00      1.00      1.00     20332



<h1>Búsqueda aleatoria de hiperparámetros para modelo K Nearest Neighbors con validación cruzada de 3 carpetas sobre el set de entrenamiento usando concurrencia</h1>

In [119]:
modelo = KNeighborsClassifier()
parametros = {'n_neighbors': range(3, 30, 2),
              'weights': ['distance', 'uniform'],
              'metric': ['euclidean', 'manhattan', 'minkowski']
             }
random_search = RandomizedSearchCV(modelo, parametros, cv=3, scoring='f1', random_state=123, n_iter=20, n_jobs=-1, verbose=1)

inicio = time.time()

random_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')
modelos.append(test_model('knn', random_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Tiempo de ejecucion:  2.54 minutos

------------------------------ knn ------------------------------

Exactitud: 0.9994097973637616
Tiempo promedio de inferencia:  32658.81300 ms

Matriz de confusion:
 [[20070     4]
 [    8   250]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       0.98      0.97      0.98       258

    accuracy                           1.00     20332
   macro avg       0.99      0.98      0.99     20332
weighted avg       1.00      1.00      1.00     20332



<h1>Búsqueda de hiperparámetros para Support Vector Machine con validación cruzada de 3 carpetas usando concurrencia</h1>

In [120]:
parametros = {'coef0': [75,100,125],
              'degree': [2,3,4],
              'kernel': ['linear','poly','rbf']
             }
modelo = svm.SVC()
grid_search = GridSearchCV(modelo, parametros, cv=3, scoring='f1', n_jobs=-1, verbose=1)

inicio = time.time()

grid_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('svm', grid_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Tiempo de ejecucion:  0.99 minutos

------------------------------ svm ------------------------------

Exactitud: 0.9989671453865827
Tiempo promedio de inferencia:  849.91932 ms

Matriz de confusion:
 [[20067     7]
 [   14   244]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       0.97      0.95      0.96       258

    accuracy                           1.00     20332
   macro avg       0.99      0.97      0.98     20332
weighted avg       1.00      1.00      1.00     20332



<h1>Entrenamiento de modelo de redes neuronales con una capa de entrada, cinco capas ocultas, dropout y una capa de salida con una funcion de activacion sigmoidal con algoritmo de optimización AdamW y funcion de perdida como entropia binaria cruzada</h1>

In [121]:
seed_value= 123
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

modelo = keras.models.Sequential()
modelo.add(keras.layers.Input(shape=(X_train.shape[1],)))
modelo.add(keras.layers.Dense(units=64, activation='relu'))
modelo.add(keras.layers.Dropout(0.2))
modelo.add(keras.layers.Dense(units=64, activation='relu'))
modelo.add(keras.layers.Dropout(0.2))
modelo.add(keras.layers.Dense(units=64, activation='relu'))
modelo.add(keras.layers.Dropout(0.2))
modelo.add(keras.layers.Dense(units=64, activation='relu'))
modelo.add(keras.layers.Dropout(0.2))
modelo.add(keras.layers.Dense(units=64, activation='relu'))
modelo.add(keras.layers.Dense(units=1, activation='sigmoid'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1000,
        decay_rate=0.5
        )

opt = keras.optimizers.AdamW(learning_rate=lr_schedule)

modelo.compile(loss='binary_crossentropy', optimizer=opt, metrics=[
        keras.metrics.F1Score(name='f1'),
        keras.metrics.BinaryAccuracy(name='accuracy')
        ])

early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

inicio = time.time()

modelo.fit(X_train, y_train.reshape(-1, 1), epochs=20,
        batch_size=128,
        callbacks=[early_stopping],
        verbose = 1
        )

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('nn', modelo, X_test, y_test))

Epoch 1/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9939 - f1: 0.0184 - loss: 0.0385
Epoch 2/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9973 - f1: 0.0191 - loss: 0.0106
Epoch 3/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9981 - f1: 0.0197 - loss: 0.0074
Epoch 4/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9984 - f1: 0.0204 - loss: 0.0061
Epoch 5/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9986 - f1: 0.0208 - loss: 0.0053
Epoch 6/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9985 - f1: 0.0210 - loss: 0.0054
Epoch 7/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9985 - f1: 0.0215 - loss: 0.0055
Epoch 8/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9985 - f1: 0.0221 - loss: 0.0058
Epoch 9/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9987 - f1: 0.0221 - loss: 0.0051
Epoch 10/20
636/636 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9987 - f1: 0.0222 - loss: 0.0047
Epoch 11/

<h1>Búsqueda de hiperparámetros para modelo Extreme Gradient Boosting con validación cruzada de 3 carpetas usando concurrencia</h1>

In [122]:
modelo = xgb.XGBClassifier(objective='binary:logistic', seed=123, eval_metric='logloss')

parametros = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [i for i in range(3,10,2)],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

grid_search = GridSearchCV(modelo, parametros, cv=3, scoring='f1', n_jobs=1, verbose=1)

inicio = time.time()

grid_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('xgb', grid_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 96 candidates, totalling 288 fits
Tiempo de ejecucion:  1.98 minutos

------------------------------ xgb ------------------------------

Exactitud: 0.9997540822349007
Tiempo promedio de inferencia:  99.99514 ms

Matriz de confusion:
 [[20074     0]
 [    5   253]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       1.00      0.98      0.99       258

    accuracy                           1.00     20332
   macro avg       1.00      0.99      1.00     20332
weighted avg       1.00      1.00      1.00     20332



<h1>Búsqueda aleatoria de hiperparámetros para modelo Random Forest con validación cruzada de 3 carpetas usando concurrencia</h1>

In [123]:
modelo = RandomForestClassifier(random_state=123)

parametros = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'class_weight': ['balanced', None]
}

random_search = RandomizedSearchCV(modelo, parametros, cv=3, n_iter=100, scoring='f1', n_jobs=-1, verbose=1)

inicio = time.time()

random_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('rf', random_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Tiempo de ejecucion:  2.08 minutos

------------------------------ rf ------------------------------

Exactitud: 0.9997540822349007
Tiempo promedio de inferencia:  350.00086 ms

Matriz de confusion:
 [[20073     1]
 [    4   254]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20074
         1.0       1.00      0.98      0.99       258

    accuracy                           1.00     20332
   macro avg       1.00      0.99      1.00     20332
weighted avg       1.00      1.00      1.00     20332



<h1>Búsqueda de hiperparámetros para modelo Gaussian Naive Bayes con validación cruzada de 3 carpetas usando concurrencia</h1>

In [124]:
parametros = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

modelo = GaussianNB()
grid_search = GridSearchCV(modelo, parametros, cv=3, scoring='f1', n_jobs=-1, verbose=1)

inicio = time.time()

grid_search.fit(X_train, y_train)

print(f'Tiempo de ejecucion: {(time.time() - inicio) / 60 : .2f} minutos')

modelos.append(test_model('gnb', grid_search.best_estimator_, X_test, y_test))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Tiempo de ejecucion:  0.11 minutos

------------------------------ gnb ------------------------------

Exactitud: 0.980326578792052
Tiempo promedio de inferencia:  260.01930 ms

Matriz de confusion:
 [[19678   396]
 [    4   254]]

Reporte:
               precision    recall  f1-score   support

         0.0       1.00      0.98      0.99     20074
         1.0       0.39      0.98      0.56       258

    accuracy                           0.98     20332
   macro avg       0.70      0.98      0.77     20332
weighted avg       0.99      0.98      0.98     20332



<h1>Despliegue y entrenamiento extra del mejor modelo según F1 Score</h1>

In [125]:
modelo = sorted(modelos, key=lambda x: x[2])[-1]

# Seleccionar por tiempo de inferencia
# modelo = sorted(modelos, key=lambda x: x[-1])[0]

modelo[1].fit(X_test, y_test)
if (modelo[0] == 'ann'):
    modelo[1].save(f'modelo_{modelo[0]}.keras')
else:
    joblib.dump(modelo[1], f'modelo_{modelo[0]}.pkl')
print(f'Mejor modelo: {modelo[0]}')
print(f'F1 Score: {modelo[2] : .3f}')

Mejor modelo: rf
F1 Score:  0.990


<h1>Ejemplo de uso sobre nuevos datos, se obtiene "output.txt" con los registros clasificados como ataques</h1>

In [126]:
import os, joblib
import numpy as np
from preprocessing import *

try:
    modelo = [f for f in os.listdir(os.curdir) if f.startswith('modelo')][0]
    modelo = joblib.load(modelo) if modelo.endswith('.pkl') else keras.models.load_model(modelo)
    preprocesador = joblib.load('preprocesador.pkl')
except Exception as e:
    raise RuntimeError('Archivo no encontrado')

df2 = pd.read_csv('testing_ataques.csv', header=None)
X = preprocesador.transform(df2)
if hasattr(modelo, 'n_features_in_') and modelo.n_features_in_ < X.shape[1]:
    result = (modelo.predict(PCA(n_components=modelo.n_features_in_).fit_transform(X))>=0.5).astype(np.float32)
else:
    result = (modelo.predict(X)>=0.5).astype(np.float32)
with open('output.txt', mode='w', encoding='utf-8') as f:
    f.write(','.join([str(el+1) for el in np.where(result.astype(int) == 1)[0]]))
    f.close()

<h1>Al ser un experimento realista no se verifican resultados pero los resultados reales están en el archivo "ataques.txt"</h1>